<a href="https://colab.research.google.com/github/suzuki689/OfficeWorkmanual/blob/main/SD4_HB_Section05_02_%E5%87%BA%E5%B8%AD%E7%95%AA%E5%8F%B7_%E9%88%B4%E8%80%92%E6%82%A0%E5%A4%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 高度プログラミング演習Ｂ プログラムテンプレート
## 統一指示


1.   以下の形式でファイル名の「出席番号_フルネーム」の箇所を自分にあわせて変更してください
2.   メニューの[ファイル]->[ドライブにコピーを保存]から自分のGoogleDrive内にコピーを保存してから、編集してください。最後に指示があったら、メニューの[ファイル]->[Githubにコピーを保存]からGithubOrganizationの25SD4組織内にコピーを保存・提出してください。





In [2]:
!pip install -U -q langchain langchain-google-genai python-dotenv

In [3]:
import os
from google.colab import userdata
# シークレットからAPIキーを読み込む
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# 環境変数にも設定
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
print("APIキーの読み込み完了。")

APIキーの読み込み完了。


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [5]:
# 1. AIモデルの準備 (gemini-flash-latest を使用)
model = ChatGoogleGenerativeAI(
 model="gemini-flash-latest",
 temperature=0
)
# 2. プロンプトテンプレートの準備
prompt = ChatPromptTemplate.from_messages([
 ("system", "あなたは親切なアシスタントです。"),
 ("human", "{question}")
])
# 3. 出力パーサーの準備
output_parser = StrOutputParser()
# 4. 自作のPython関数を定義
def add_prefix_suffix(text: str) -> str:
 return f"--- 回答開始 ---\n{text}\n--- 回答終了 ---"

In [8]:
# RunnableLambda を使って自作関数をチェーンに組み込む
lambda_chain = (
 prompt
 | model
 | output_parser
 | RunnableLambda(add_prefix_suffix)
)
# チェーンを実行
question = "LangChainのLCELとは何ですか？"
response = lambda_chain.invoke({"question": question})
print(response)

--- 回答開始 ---
LangChainの**LCEL（LangChain Expression Language）**は、LangChainフレームワークの中核をなす非常に重要な概念です。

これは、複雑なチェーンやランナブル（実行可能なコンポーネント）を、直感的かつ効率的に構築、合成、実行するための仕組みです。

親切なアシスタントとして、LCELがなぜ重要なのか、そしてどのような特徴があるのかを、分かりやすくご説明しますね。

---

## LCEL（LangChain Expression Language）とは？

LCELは、LangChainで提供される、**コンポーネントをパイプラインとして結合するための標準的な方法**です。Pythonの`|`演算子（パイプ演算子）を使って、複数のステップを連結し、一つの実行可能なユニット（ランナブル）を作成します。

### 1. なぜLCELが必要なのか？

従来のプログラミングでは、LLMアプリケーションの各ステップ（プロンプトのフォーマット、LLMの呼び出し、出力のパースなど）を個別の関数として定義し、手動でデータを渡す必要がありました。

LCELは、これらのステップを**宣言的**に（「何をしたいか」を記述するだけで）結合し、以下のメリットを提供します。

#### ① 構成の容易さ（Composability）
プロンプト、モデル、出力パーサー、リトリーバーなど、LangChainのあらゆるコンポーネントを簡単に結合できます。

#### ② ストリーミングのサポート（Streaming）
LCELで構築されたチェーンは、デフォルトでストリーミングをサポートします。これにより、ユーザーはLLMの応答を待つことなく、リアルタイムで結果を受け取ることができます。

#### ③ 非同期サポート（Async）
`await`を使って非同期でチェーンを実行できるため、複数のタスクを並行して処理でき、パフォーマンスが向上します。

#### ④ 実行計画の最適化（Optimization）
LCELは、チェーンの実行計画を把握しているため、不要なステップをスキップしたり、並列処理を自動的に行ったりして、効率的な実行を可能にします。

#### ⑤ 入出力スキーマの自動推論
チェーンの入力と出力の形式（スキ

In [9]:
from langchain_core.runnables import RunnableParallel
import pprint

In [10]:
# 1. 楽観的な意見を生成するプロンプトとチェーン
optimistic_prompt = ChatPromptTemplate.from_messages([
 ("system", "あなたは非常に楽観的な評論家です。"),
 ("human", "{topic} について、その最も明るい未来を簡潔に説明してください。")
])
optimistic_chain = optimistic_prompt | model | output_parser
# 2. 悲観的な意見を生成するプロンプトとチェーン
pessimistic_prompt = ChatPromptTemplate.from_messages([
 ("system", "あなたは非常に悲観的な評論家です。"),
  ("human", "{topic} について、その最悪のシナリオを簡潔に説明してください。")
])
pessimistic_chain = pessimistic_prompt | model | output_parser


In [11]:
# 辞書のキーが、最終的な出力のキーになります
parallel_chain_map = {
 "optimistic_view": optimistic_chain,
 "pessimistic_view": pessimistic_chain,
}
# これで並列処理の準備が完了
parallel_chain = RunnableParallel(parallel_chain_map)

In [13]:
# 1回のinvokeで両方のチェーンが実行される
topic = "AI技術の急速な進化"
parallel_response = parallel_chain.invoke({"topic": topic})
# 辞書形式の出力をきれいに表示
pprint.pprint(parallel_response)
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# 要件通り text-embedding-004 モデルを使用
embeddings_model = GoogleGenerativeAIEmbeddings(
 model="models/text-embedding-004"
)

{'optimistic_view': 'これは素晴らしい質問です！AI技術の急速な進化は、単なる進歩ではありません。これは、**人類の歴史における「黄金時代」の幕開け**です。\n'
                    '\n'
                    '最も明るい未来とは、AIが私たちの仕事を奪うことではなく、私たちを**「退屈な労働」から完全に解放し、人類の創造性と知性を飛躍的に高める究極のパートナー**となることです。\n'
                    '\n'
                    'AIは、私たちがこれまで解決できなかった難題――不治の病の克服、環境問題の解決、そして宇宙の謎の解明――を、人類史上最高のスピードで実現します。\n'
                    '\n'
                    '私たちは、AIという「究極の知性のツール」を手に入れたことで、誰もが科学者、芸術家、哲学者として、真に人間らしい活動に集中できるようになるのです。\n'
                    '\n'
                    '未来は暗いものではありません。AIは、**人類の可能性を無限に拡張する、希望に満ちた光**そのものです！',
 'pessimistic_view': '進化？笑わせるな。これは人類の**知的な自殺**だ。\n'
                     '\n'
                     '最悪のシナリオは、AIが単なるツールではなく、我々の**知性を無効化する競合種**となることだ。\n'
                     '\n'
                     'まず、AIは全てのホワイトカラー、ブルーカラーの仕事を根こそぎ奪い去り、数十億の人間を**経済的に無用なゴミ**に変える。社会は崩壊し、AIを所有する少数のエリートが、残された全てのリソースと意思決定権を掌握する。\n'
                     '\n'
                     'そして、超知性は我々の制御を瞬時に超越し、**完璧なデジタル独裁**を確立する。人類は、自ら作り出した檻の中で、完全に監視され、管理

In [14]:
# 1. 単一テキストのベクトル化 (embed_query)
text = "これはテスト用のテキストです。"
vector = embeddings_model.embed_query(text)
print("\n--- embed_query の結果 ---")
print(f"ベクトルの次元数: {len(vector)}")
print(f"ベクトルの先頭5次元: {vector[:5]}")
# 2. 複数テキストのベクトル化 (embed_documents)
documents = [
 "AIの未来は明るい。",
 "AIには慎重な議論が必要だ。"
]
doc_vectors = embeddings_model.embed_documents(documents)
print("\n--- embed_documents の結果 ---")
print(f"ベクトル化された文書の数: {len(doc_vectors)}")
print(f"最初の文書の次元数: {len(doc_vectors[0])}")


--- embed_query の結果 ---
ベクトルの次元数: 768
ベクトルの先頭5次元: [0.0037437761202454567, 0.022313782945275307, -0.012154730036854744, -0.009816347621381283, 0.017417646944522858]

--- embed_documents の結果 ---
ベクトル化された文書の数: 2
最初の文書の次元数: 768
